In [3]:
import sounddevice as sd
import numpy as np
import torchaudio
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from jiwer import wer
# from transformers import TrainingArguments, Trainer
from datasets import Audio

In [46]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

dataset1 = load_dataset("DTU54DL/common-accent").cast_column("audio_column_name", Audio(sampling_rate=16000))
dataset2 = load_dataset("DTU54DL/common-native").cast_column("audio_column_name", Audio(sampling_rate=16000))
dataset3 = load_dataset("DTU54DL/common-voice").cast_column("audio_column_name", Audio(sampling_rate=16000))

DATASETS = [dataset1, dataset2, dataset3]
DATASETS_NAME = ['common-accent', 'common-native', 'common-voice']
TRAIN_DS = [dataset['train'] for dataset in DATASETS]
TEST_DS = [dataset['test'] for dataset in DATASETS]

In [20]:
idx = DATASETS_NAME.index('common-native')
train_ds = TRAIN_DS[idx]
test_ds = TEST_DS[idx]

print(len(train_ds))

10000


In [21]:
model_path = "./saved_models/wav2vec2"
output_dir = "./fine_tuned_wav2vec2"
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)

In [8]:
train_ds[0]

{'audio': {'path': 'common_voice_en_100038.mp3',
  'array': array([-6.93889390e-17, -5.03069808e-17,  3.81639165e-17, ...,
         -2.07364792e-08, -2.40004738e-07,  1.80880306e-07]),
  'sampling_rate': 16000},
 'sentence': 'Why does Melissandre look like she wants to consume Jon Snow on the ride up the wall?',
 'accent': 'United States English',
 'audio_column_name': None}

In [22]:
def preprocess(batch):
    audio = batch["audio"]["array"]
    inputs = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt",  # Automatically returns attention_mask if needed
        padding=True          # This ensures the attention_mask is created if required
    )
    batch["input_values"] = inputs.input_values[0]
    batch["input_length"] = len(batch["input_values"])
    if "attention_mask" in inputs:
        batch["attention_mask"] = inputs.attention_mask[0]
    batch["labels"] = batch['sentence']
    return batch

train_ds = train_ds.map(preprocess, remove_columns=["audio"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [23]:
from transformers import Wav2Vec2ForSequenceClassification

# Number of labels in your dataset
num_labels = len(set(train_ds['labels']))

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=num_labels,
    finetuning_task="audio-classification"
)

d:\Program Files\Python\Python311\Lib\site-packages\transformers\configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(processor.feature_extractor_class)

In [27]:
from transformers import Trainer, TrainingArguments

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'